# Build a revenue dashboard using PixieApps


## Introduction 

This notebook is divided into three parts. In part 1 we capture clickstream events that indicate that a customer has made a purchase and  enrich these events by adding geolocation information. Using this information we aggregate the total revenue by ZIP code (for US-based transactions) or country code (for international transactions) and periodically write totals (representing revenue) to generic cloud storage (represented in this scenario by a bucket in Cloud Object Storage). In part 2 we add the aggregated data to a Watson Data Platform catalog and in part 3 we use a PixieApp to browse the catalog using the Watson Data Platform API and visualize revenue information.


<img src="https://raw.githubusercontent.com/IBMCodeLondon/localcart-workshop/master/images/part_3.png"></img>


[Part 1](#part1): Capturing and aggregating clickstream events<br>
[Part 2](#part2): Managing data access using the Watson Data Platform catalog<br>
[Part 3](#part3): Visualizing clickstream events using a PixieApp<br>


This notebook runs on Python 2 with Spark 2.1.


<a id="part1"></a>
***
# Part 1: Capturing and aggregating clickstream events 
***

<img src="https://raw.githubusercontent.com/IBMCodeLondon/localcart-workshop/master/images/dynamic_analysis_with_apixieapps_part_1.png"></img>


## Part 1 table of contents

[1.1 Import a streams flow](#import_flow) <br>
[1.2 Customize the streams flow](#customize_flow) <br>
[1.3 Run the flow](#run_flow)<br>

<a id="import__flow"></a>
***

## 1.1 Import a streams flow

In this notebook you'll import and customize a streams flow that aggregates sales transactions and writes them to Cloud Object Storage.

First

1. Download https://raw.githubusercontent.com/IBMCodeLondon/localcart-workshop/master/streams_flows/revenue_by_state_or_country.stp to your local machine. This file contains the streams flow definition you'll be working with.

Next, complete the following steps in IBM Watson Data Platform:

1. [Select (or create) a project](https://dataplatform.ibm.com/projects?context=analytics) that you want to contain the streams flow. Note that this project must be attached to Cloud Object Storage and not Object Storage (Swift).
1. Click the **Assets** tab and scroll to the _Streams flows_ section. 
 > If no section with the name is displayed the selected project is not attached to Cloud Object Storage.
 
 > If there is still no section with the name in the selected project go [here](https://dataplatform.ibm.com/streams/pipelines) to find the *Streams flows*. Or copy your projectid into this URL `https://dataplatform.ibm.com/streams/pipelines?projectid=<projectid>` (you can find the projectid from the URL of your project page: for example `https://dataplatform.ibm.com/projects/<projectid>/overview?context=wdp/` )
 
1. Click **+ New streams flow**.
1. In the _New Streams Flow_ window, 
  1. Select **from file**
  1. Select an existing Streaming Analytics service or create a new one (choosing the _Lite_ plan, which is free.) 
  1. Browse to the streams flow file you've downloaded. 
   > The flow name and description are populated for you. You can change the default if desired.
  1. Click **Create**. Wait for the import to complete.
  
1. Review the flow. It comprises of one source operator (Message Hub), a [Python] code operator (retrieving customer geolocation information from the Cloud), two filter operators (separating US transactions from international transactions), two aggregation operators (one for each major geography, calculating the revenue) and two Cloud Object Storage target operators, saving the aggregated data for later processing.
 
 > You'll notice that the run button is disabled, because the flow it is not yet properly configured for your environment. 

1. To identify the issues, click the highlighted notification icon on the right hand side.
<img src="https://raw.githubusercontent.com/ibm-watson-data-lab/localcart-at-think-conf/master/images/dynamic_analysis_pixieapp_part1_flow_issues.png"></img>
1. Click on the notification to open the canvas and expand the highlighted error list icon.
 > Note that the (Message Hub) source operator and the two Cloud Object Storage target operators are tagged as invalid. This is expected because they are associated with connections that you don't have access to.

<a id="customize_flow"></a>
***

## 1.2 Customize the streams flow


#### Resolve the Message Hub operator issue
1. Open the Message Hub operator.
 > Note that no connection and no topic are assigned to it.
1. Select your existing Message Hub connection or create a new one. 
1. The **logout_with_purchase** topic should be pre-selected, capturing completed sales events.
1. Review, but do not change the schema. Note the `customer_id` and `total_price_of_basket`. Only these two attributes are consumed by the flow. We kept the other attributes in the schema definition only for illustrative purposes.
1. Save the streams flow.
> The Source Message Hub operator should no longer be flagged as invalid.
 
#### Resolve the Cloud Object Storage operator issues

1. Open the first _Cloud Object Storage_ operator.
 > Note that no connection is assigned to the operator because your Watson Data Platform environment is different from the environment where the flow was created.
1. Select your existing Cloud Object Storage connection or create a new one. 
1. Customize the file path, which defines where operator will write the output to
  1. Open the data asset selector and select an existing bucket.
   > Don't choose an existing object from the list. You'll specify a new generic name in the next step; you will need to provide a globally unique name for your bucket.
  1. Append `/us_revenue_%TIME` to the file path, to specify the object name pattern
   > `%TIME` will be replaced with a timestamp. Your path should look as follows: `/my-existing-bucket/us_revenue_%TIME.csv`
1. Review the other settings but do not make any other changes.
 > Pay attention to the file writing policy, which defines how frequently aggregated data is written to storage.
1. Save the streams flow.
 > The first Cloud Object Storage operator should no longer be flagged as invalid.

 ***

1. Open the second _Cloud Object Storage_ operator
1. Select your existing Cloud Object Storage connection. 
1. Customize the file path, which defines where operator will write the output to.
  1. Open the data asset selector and select an existing bucket.
  1. Append `/foreign_revenue_%TIME` to the file path, to specify the object name pattern
   > `%TIME` will be replaced with a timestamp. Your path should look as follows: `/my-existing-bucket/foreign_revenue_%TIME.csv`
1. Review the other settings but do not make any other changes.
1. Save the streams flow.
> The streams flow should now be valid. You should see meaningful data in your object storage bucket after 5 minutes. This is due to the default settings in the aggregation: a 5-minute, tumbling window. Feel free to change this setting if you'd prefer to speed things up.


<a id="run_flow"></a>
***

## 1.3 Run the flow

1. Run the customized flow. After a minute or two data should be streaming from the source to the targets.

   <img src="https://raw.githubusercontent.com/ibm-watson-data-lab/localcart-at-think-conf/master/images/dynamic_analysis_pixieapp_part1_running.png"></img>

1. Wait until at least one data file containing revenue information for US transactions and international transactions has been written to the specified Cloud Object Storage bucket before continuing.



<a id="part2"></a>
***
# Part 2: Managing data access using the Watson Data Platform Data Catalog
***
<p>
With Data Catalog, an optional add-on application for the Watson Data Platform you can easily find and share data, regardless of the location (e.g. cloud storage, database) or format (e.g. csv). [[Learn more...](https://dataplatform.ibm.com/docs/content/catalog/overview-dc.html?audience=wdp)] </p>
<p>In this second part of the notebook you'll create a new data catalog and add the aggregated revenue files to it.</p>

<img src="https://raw.githubusercontent.com/ibm-watson-data-lab/localcart-at-index-conf/master/images/dynamic_analysis_with_apixieapps_part_2.png"></img>


## Part 2 table of contents

[2.1 Create a data catalog](#part2_create_catalog) <br>
[2.2 Add assets to the Data Catalog](#part2_add_assets_to_catalog) <br>

<a id="part2_create_catalog"></a>
## 2.1 Create a Data Catalog
1. In Watson Data Platform open the catalogs view (**Catalog** > **View All Catalogs**) or click this [direct link](https://dataplatform.ibm.com/data/catalogs?context=analytics).
   > The catalog menu entry is only displayed if the IBM Data Catalog application was added to your Watson Data Platform. To add the app, click on your avatar icon on the upper right hand side and choose from the menu "Add Other apps".
2. Add a catalog.
3. In this example we will be adding the revenue assets generated from the stream defined in the first part. Be sure to associate the Cloud Object Storage instance from part 1 in your new catalog.
 > When adding your Cloud Object Storage connection, note that the "Access Key" and "Secret Key" fields are **not** required. These options are only for those [using S3-style authentication](https://console.bluemix.net/docs/services/cloud-object-storage/iam/service-credentials.html#service-credentials) on IBM's Cloud Object Stroage service.

<a id="part2_add_assets_to_catalog"></a>
## 2.2 Add Assets to the Data Catalog

Assets are resources such as data files and connections. The data catalog only stores metadata about those assets, not the actual content.

#### Add Cloud Object Storage connection to the Data Catalog

1. In Watson Data Platform open the **Projects** menu and select the project you've used in part 1.
2. Select the actions button next to your Cloud Object Storage instance and choose **Publish** from the menu.
3. Choosing the catalog you've created in the previous step as the target, publish the connection.

#### Add revenue files to the Data Catalog

1. In Watson Data Platform open the **Catalog** menu and select the desired catalog.
2. Repeat the following steps for at least one US revenue file and one foreign revenue file.
   1. Add **Connected data** to the catalog.
   2. As source, select the connection you've published to the catalog.
   3. Choose the Cloud Object Storage bucket to which the aggregated revenue files are being written by the streams flow.
   4. Select the revenue file you would like to add to your catalog. 
   5. Specify the name of the asset. Important: the name must match the name of the file. For US revenue files the name must be *us_revenue_DATE_TIME*. For foreign revenue files the name must be *foreign_revenue_DATE_TIME*.
   > Note that you can tag assets. Tags serve two purposes: they make it easier to locate data and support data governance rules.
3. Your catalog should now contain data assets that authorized users can access using the Watson Data Platform apps or programatically, as illustrated in the next part.

<img src="https://raw.githubusercontent.com/ibm-watson-data-lab/localcart-at-index-conf/master/images/dynamic_analysis_with_pixieapps_part2_connected_data.png"></img>


<a id="part3"></a>
***
# Part 3: Visualizing clickstream events using a PixieApp
***

In the third part of this scenario you will learn how to build a PixieApp dashboard from data assets stored in a Watson Data Platform Catalog.

<img src="https://raw.githubusercontent.com/ibm-watson-data-lab/localcart-at-index-conf/master/images/dynamic_analysis_with_apixieapps_part_3.png"></img>


## Part 3 table of contents

[3.1 Prerequistes](#part3_prerequisites) <br>
[3.2 Define Watson Data Platform API helpers](#part3_api_helpers) <br>
[3.3 Define misc helpers](#part3_misc_helpers) <br>
[3.4 Customize PixieApp settings](#part3_customize) <br>
[3.5 Define a custom PixieDust renderer](#part3_custom renderer) <br>
[3.6 Implement a PixieApp dashboard](#part3_pixieapp) <br>

<a id="part3_prerequisites"></a>
### 3.1 Prerequisites


#### Generate IBM Cloud API Key
1. Log in to your IBM Cloud account
2. Click **Manage** in the toolbar, highlight **Account**, and click **Users**
3. In the navigation panel click **Platform API Keys**
4. Click **Create**

#### Google Maps API Key
You can obtain an API key from the [Google API Console](https://code.google.com/apis/console/).

After [getting the API key](https://support.google.com/googleapi/answer/6158862?hl=en&ref_topic=7013279), it must be enabled for the [Google Maps JavaScript API and the Geocoding API](https://console.developers.google.com/projectselector/apis/library).

In [ ]:
import json
import requests

<a id="part3_api_helpers"></a>

### 3.2 Define Watson Data Platform API helpers

In [ ]:
# Watson Data Platform API base URL (no change required)
wdp_api_url = 'https://api.dataplatform.ibm.com'

The following functions use the Watson Data Platform API to list catalogs and download assets. The Watson Data Platform API requires an IBM Cloud API Key as mentioned above. Below we define the following functions that will be used in our dashboard:

1. **is_valid_access_token** - checks if a previously generated access token is still valid
2. **get_access_token** - uses your IBM Cloud API Key to generate a temporary access token for working with the Watson Data Platform API
3. **get_catalogs** - gets a list of catalogs that you have access to based on the IBM Cloud API key
4. **get_data_assets** - gets a list of data assets in a catalog
5. **get_data_asset** - gets a specific data asset
6. **get_connection** - gets a connection from the *connection_id* (in this case the *connection_id* is retrieved from the data asset)

In [ ]:
# check if access token is valid by trying to get a list of catalogs
def is_valid_access_token(access_token):
    url = '{0}/v2/catalogs'.format(wdp_api_url)
    headers = {
        'Authorization': 'Bearer {}'.format(access_token)
    }
    response = requests.get(url, headers=headers)
    return response.status_code == 200

def get_access_token(token_url, api_key):
    url = '{0}?apikey={1}&grant_type=urn:ibm:params:oauth:grant-type:apikey'.format(token_url, api_key)
    response = requests.post(url)
    if response.status_code == 200:
        response_obj = json.loads(response.text)
        if 'access_token' in response_obj.keys():
             return response_obj['access_token']
    return None

def get_catalogs(access_token):
    url = '{0}/v2/catalogs'.format(wdp_api_url)
    headers = {
        'Authorization': 'Bearer {}'.format(access_token)
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        response_obj = json.loads(response.text)
        if 'catalogs' in response_obj.keys():
            return response_obj['catalogs']
    return None

def get_data_assets(access_token, catalog_id):
    url = '{0}/v2/asset_types/asset/search?catalog_id={1}'.format(wdp_api_url, catalog_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {}'.format(access_token)
    }
    body = {
      'query': 'asset.asset_type:data_asset'
    }
    response = requests.post(url, headers=headers, json=body)
    if response.status_code == 200:
        response_obj = json.loads(response.text)
        if 'results' in response_obj.keys():
            return response_obj['results']
    return None

def get_data_asset(access_token, asset_id, catalog_id):
    url = '{0}/v2/data_assets/{1}?catalog_id={2}'.format(wdp_api_url,asset_id,catalog_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {}'.format(access_token)
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.text)
    return None

def get_connection(access_token, connection_id, catalog_id):
    url = '{0}/v2/connections/{1}?catalog_id={2}'.format(wdp_api_url,connection_id,catalog_id)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer {}'.format(access_token)
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        response_obj = json.loads(response.text)
        if 'entity' in response_obj.keys():
            return response_obj['entity']
    return None

<a id="part3_misc_helpers"></a>

### 3.3 Define misc helper functions

The following helper functions are used by the dashboard:
1. **download_file** - downloads a file from Cloud Object Storage
2. **is_valid_google_maps_api_key** - validates a Google Maps API key

In [ ]:
def download_file(access_token, url):
    headers = {
        'Authorization': 'Bearer {}'.format(access_token)
    }
    return requests.get(url, headers=headers)

def is_valid_google_maps_api_key(api_key):
    response = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key="+api_key)
    if (response.status_code == 200 and response.json()['status'] != 'OK') \
        or response.status_code != 200:
            return False
    return True

<a id="part3_customize"></a>

### 3.4 Customize PixieApp settings
You can define the following variables to be used by the dashboard. If these variables are not defined the dashboard will prompt the user to enter them:
1. **app_ibm_cloud_platform_api_key**
2. **app_google_maps_api_key**
3. **app_catalog_name**

In [ ]:
app_ibm_cloud_platform_api_key = None
app_google_maps_api_key = None
app_catalog_name = None

<a id="part2_custom renderer"></a>

### 3.5 Create a custom 3D Bar chart rendering function for PixieDust

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pylab
def render_us_3D_chart(dashboard):
    dpi = pylab.gcf().get_dpi()
    fig = plt.figure(
        dpi=96, facecolor='w',
        figsize=( int(dashboard.getPreferredOutputWidth()/dpi), int(dashboard.getPreferredOutputHeight()/dpi) )
    )
    ax = fig.add_subplot(111, projection='3d')
    pdf = dashboard.pdf_us_filtered.groupby('us_state')\
        .agg({'us_state': 'size', 'revenue': 'sum'}) \
        .rename(columns={'us_state':'transactions'}) \
        .reset_index()

    num_elements = len(pdf)
    xpos = [i for i in range(num_elements)]
    rev = pdf['revenue'].values.tolist()
    ypos = [0]
    for i, rev in enumerate(pdf['revenue'].values.tolist()):
        if i < num_elements - 1:
            ypos.append(rev + ypos[i])
    zpos = [0] * num_elements
    dx = [1] * num_elements
    dy = pdf['revenue'].values.tolist()
    dz = pdf['transactions'].values.tolist()

    ax.set_xlabel('US States')
    ax.set_xlim3d(auto=True)
    ax.axes.set_xticks(xpos)
    ax.axes.set_xticklabels(pdf['us_state'].values.tolist())

    ax.set_ylabel('Revenue')
    ax.set_zlabel('Number of transactions')

    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color='#00ceaa')
    plt.show()

<a id="part3_pixieapp"></a>
### 3.6 Implement a PixieApp dashboard

In [ ]:
from pixiedust.utils.userPreferences import *
from pixiedust.display.app import *
from datetime import datetime
import io
import pandas
import re
import requests

user_pref_access_token = 'advolaunch.wdp.catalog.access_token'
user_pref_google_maps_api_key = 'advolaunch.wdp.catalog.google_maps_api_key'
regex_foreign = re.compile('(foreign_revenue_[0-9]{8}_[0-9]{6}).*')
regex_us = re.compile('(us_revenue_[0-9]{8}_[0-9]{6}).*')
date_format_foreign = 'foreign_revenue_%Y%m%d_%H%M%S'
date_format_us = 'us_revenue_%Y%m%d_%H%M%S'

@PixieApp
@Logger()
class CatalogDashboard(object):
    
    @route()
    def default(self):
        access_token_valid = False
        access_token = getUserPreference(user_pref_access_token)
        if access_token is not None:
            access_token_valid = is_valid_access_token(access_token)
        if access_token_valid:
            self.access_token = access_token
            return self.show_catalogs()
        else:
            try:
                self.api_key = app_ibm_cloud_platform_api_key
            except NameError:
                  return self.show_api_key()
            else:
                if self.api_key is None:
                    return self.show_api_key()
                else:
                    return self._get_access_token()

    @route(route_get_access_token="true")
    def _get_access_token(self):
        self.access_token = get_access_token('https://iam.ng.bluemix.net/oidc/token', self.api_key)
        if self.access_token is not None:
            setUserPreference(user_pref_access_token, self.access_token)
        return self.show_catalogs()
    
    @route(route_set_google_maps_api_key="true")
    def set_google_maps_api_key(self):
        if self.google_maps_api_key is not None:
            setUserPreference(user_pref_google_maps_api_key, self.google_maps_api_key)
        return self.show_catalogs()

    @route(route_select_catalog="true")
    def select_catalog(self):
        self.route_select_catalog="false" # reset route
        self.connection_cache = {}
        self.pdf_foreign = None
        self.pdf_us = None
        self.pdf_us_filtered = None
        data_assets = get_data_assets(self.access_token, self.catalog_id)
        for data_asset in data_assets:
            data_asset_name = data_asset['metadata']['name']
            match = regex_foreign.match(data_asset_name)
            if match is not None:
                pdf = self.download_data_asset(self.access_token, data_asset['metadata']['asset_id'], self.catalog_id)
                if pdf is None:
                    continue
                pdf["date"] = datetime.strptime(match.group(1), date_format_foreign)
                if self.pdf_foreign is None:
                    self.pdf_foreign = pdf
                else:
                    self.pdf_foreign = pandas.concat([self.pdf_foreign,pdf])
            else:
                match = regex_us.match(data_asset_name)
                if match is not None:
                    pdf = self.download_data_asset(self.access_token, data_asset['metadata']['asset_id'], self.catalog_id)
                    if pdf is None:
                        continue
                    pdf["date"] = datetime.strptime(match.group(1), date_format_us)
                    if self.pdf_us is None:
                        self.pdf_us = pdf
                    else:
                        self.pdf_us = pandas.concat([self.pdf_us,pdf])
        # handle no data
        if self.pdf_us is None and self.pdf_foreign is None:
            return self.show_empty_dashboard()
        elif self.pdf_us is None:
            self.pdf_us = pandas.DataFrame({'us_zip' : [], 'us_state': [], 'revenue': [], 'date': []})
        elif self.pdf_foreign is None:
            self.pdf_foreign = pandas.DataFrame({'country_code': [], 'revenue': [], 'date': []})
        # sort
        if self.pdf_us is not None:
            self.pdf_us.sort_values(by=['date', 'us_state'])
        if self.pdf_foreign is not None:
            self.pdf_foreign.sort_values(by=['date', 'country_code'])
        # revenue
        revenue_us_f = self.pdf_us['revenue'].sum()
        revenue_foreign_f = self.pdf_foreign['revenue'].sum()
        self.revenue_us = '${:,.2f}'.format(revenue_us_f) 
        self.revenue_foreign = '${:,.2f}'.format(revenue_foreign_f)
        self.revenue_total = '${:,.2f}'.format(revenue_us_f + revenue_foreign_f)
        #
        self.selected_states = []
        self.selected_us_chart = 'bar'
        self.selected_foreign_chart = 'bar'
        return self.show_dashboard()
    
    @route(route_load_us_state_revenue="*")
    def load_us_state_revenue(self):
        if self.pdf_us_filtered is not None and len(self.selected_states) > 0:
            return '<span>${:,.2f}</span>'.format(self.pdf_us_filtered['revenue'].sum()) 
        else:
            return '<span>N/A</span>'
        
    @route(route_load_us_filters="*")
    def load_us_filters(self):
        return self.get_us_filters_html()

    @route(route_load_us_chart="*")
    def load_us_chart(self):
        if len(self.selected_states) == 0:
            self.selected_us_chart = 'bar'
            self.pdf_us_filtered = self.pdf_us
        else:
            self.pdf_us_filtered = self.pdf_us[self.pdf_us['us_state'].isin(self.selected_states)]
        selected_states_str = 'None'
        if self.selected_states is not None:
            selected_states_str = ','.join(self.selected_states)
        
        if self.selected_us_chart == '3D':
            return self.show_us_3D_chart()

        return """
<div pd_entity="pdf_us_filtered" pd_options=\"""" + self.get_us_chart_pd_options() + """\" pd_render_onload>
</div>
        """
    
    @captureOutput
    def show_us_3D_chart(self):
        render_us_3D_chart(self)
        
    @route(route_load_foreign_chart="*")
    def load_foreign_chart(self):
        return """
<div pd_entity="pdf_foreign" pd_options=\"""" + self.get_foreign_chart_pd_options() + """\" pd_render_onload>
</div>
        """
    
    def add_state(self, state):
        self.selected_states.append(state);
        
    def remove_state(self, state):
        self.selected_states.remove(state);
        
    def set_us_chart(self, chart):
        self.selected_us_chart = chart

    def get_us_chart_pd_options(self):
        if len(self.selected_states) == 0:
            return "handlerId=table;table_noschema=true;valueFields=revenue;keyFields=us_state;aggregation=SUM;rowCount=500;legend=true;rendererId=bokeh;clusterby="
        else:
            if self.selected_us_chart == 'line':
                return "noChartCache=true;handlerId=lineChart;valueFields=revenue;keyFields=date;aggregation=SUM;rowCount=500;legend=true;rendererId=bokeh;clusterby=us_state"
            else:
                return "noChartCache=true;handlerId=barChart;valueFields=revenue;keyFields=us_state;aggregation=SUM;rowCount=500;legend=true;rendererId=bokeh;clusterby="
        
    def set_foreign_chart(self, chart):
        self.selected_foreign_chart = chart
        
    def get_foreign_chart_pd_options(self):
        if self.selected_foreign_chart == 'line':
            return "handlerId=lineChart;valueFields=revenue;keyFields=date;aggregation=SUM;rowCount=500;timeseries=false;legend=true;rendererId=bokeh;clusterby=country_code"
        else:
            return "handlerId=barChart;valueFields=revenue;keyFields=country_code;aggregation=SUM;rowCount=500;timeseries=false;legend=true;rendererId=bokeh;clusterby="
    
    def handle_us_event(self, event_info):
        if 'us_state' in event_info.keys():
           state = event_info['us_state']
           if state in self.selected_states:
               self.remove_state(state)
           else:
               self.add_state(state)
    
    def show_api_key(self):
        return """
<div>
  <div class="form-horizontal">
    <div class="form-group">
        <div class="col-sm-2"></div>
        <div class="col-sm-5">Enter your IBM Cloud Platform API Key</div>
    </div>
    <div class="form-group">
      <label for="passcode{{prefix}}" class="control-label col-sm-2">API Key:</label>
      <div class="col-sm-5">
        <input type="text" class="form-control" id="apikey{{prefix}}">
      </div>
      <div class="col-sm-1">
        <button type="submit" class="btn btn-primary" pd_refresh>Go
          <pd_script>
self.api_key="$val(apikey{{prefix}})"
self.route_get_access_token="true"
          </pd_script>
        </button>
      </div>
    </div>
  </div>
</div>
"""
    def show_google_maps_api_key(self):
        return """
<div>
  <div class="form-horizontal">
    <div class="form-group">
        <div class="col-sm-2"></div>
        <div class="col-sm-5">Enter your Google Maps API Key</div>
    </div>
    <div class="form-group">
      <label for="passcode{{prefix}}" class="control-label col-sm-2">Key:</label>
      <div class="col-sm-5">
        <input type="text" class="form-control" id="mapsapikey{{prefix}}">
      </div>
      <div class="col-sm-1">
        <button type="submit" class="btn btn-primary" pd_refresh>Go
          <pd_script>
self.google_maps_api_key="$val(mapsapikey{{prefix}})"
self.route_get_access_token="false"
self.route_set_google_maps_api_key="true"
          </pd_script>
        </button>
      </div>
    </div>
  </div>
</div>
"""
    
    def show_catalogs(self):
        # first make sure there is a valid Google Maps API key
        # check if google maps api key defined in notebook (app_google_maps_api_key)
        # if not, then check if google maps api key stored in preferences
        # if a valid key not found then prompt user
        google_maps_api_key_valid = False
        google_maps_api_key = None
        try:
            google_maps_api_key = app_google_maps_api_key
        except NameError:
            pass
        if google_maps_api_key is None:
            google_maps_api_key = getUserPreference(user_pref_google_maps_api_key)
        if google_maps_api_key is not None:
            google_maps_api_key_valid = is_valid_google_maps_api_key(google_maps_api_key)
        if google_maps_api_key_valid:
            self.google_maps_api_key = google_maps_api_key
        else:
            return self.show_google_maps_api_key()
        # google maps api key is valid, get list of catalogs
        # if catalog_id is set (i.e. from the pulldown in the dashboard)
        # then return
        self.catalogs = get_catalogs(self.access_token)
        try:
            catalog_id = self.catalog_id
            if catalog_id is not None:
                return self.select_catalog()
        except:
            pass
        # if the catalog name was specified in the
        # notebook (app_catalog_name) then use it automatically
        catalog_name = None
        try:
            catalog_name = app_catalog_name
        except NameError:
            pass
        if catalog_name is not None:
            for catalog in self.catalogs:
                if catalog['entity']['name'] == catalog_name:
                    self.catalog_id = catalog['metadata']['guid']
                    return self.select_catalog()
        # prompt user to select catalog
        return """
<div>
  <div class="form-horizontal">
    <div class="form-group">
      <label for="org{{prefix}}" class="control-label col-sm-2">Select a catalog:</label>
      <div class="col-sm-5">
        <select class="form-control" id="catalog{{prefix}}">
        {%for catalog in this.catalogs%}
          <option value="{{catalog['metadata']['guid']}}">{{catalog['entity']['name']}}</option>
        {%endfor%}
        </select>
      </div>
      <div class="col-sm-1">
        <button type="submit" class="btn btn-primary" pd_refresh>Go
          <pd_script>
self.catalog_id="$val(catalog{{prefix}})"
self.get_access_token="false"
self.route_set_google_maps_api_key="false"
self.route_select_catalog="true"
          </pd_script>
        </button>
      </div>
    </div>
  </div>
</div>"""
        
    def show_empty_dashboard(self):
        self._addHTMLTemplateString("""
<div style="background-color: lightblue;">
  <div style="text-align: left; padding-top: 10px;">
    <select id="catalog{{prefix}}" class="form-control" style="width: auto;" pd_script="self.catalog_id='$val(catalog{{prefix}})'" pd_refresh>
    {%for catalog in this.catalogs%}
      <option value="{{catalog['metadata']['guid']}}" {%if catalog['metadata']['guid']==this.catalog_id%}selected{%endif%}>{{catalog['entity']['name']}}</option>
    {%endfor%}
    </select>
  </div>
  <div style="width: 100%; text-align: center; padding: 10px 0px 20px 0px;">
    <h3>No data available in the selected catalog.</h3>
  </div>
</div>
""")
        
    def show_dashboard(self):
        return """
<div style="text-align: left; padding-top: 10px;">
    <select id="catalog{{prefix}}" class="form-control" style="width: auto;" pd_script="self.catalog_id='$val(catalog{{prefix}})'" pd_refresh>
    {%for catalog in this.catalogs%}
      <option value="{{catalog['metadata']['guid']}}" {%if catalog['metadata']['guid']==this.catalog_id%}selected{%endif%}>{{catalog['entity']['name']}}</option>
    {%endfor%}
    </select>
  </div>
  <div style="width: 100%; text-align: center; padding: 10px 0px 20px 0px;">
    <h1>Revenue</h1>
  </div>
  <div class="tab-content container-fluid">
    <div class="row">
      <div class="col-sm-2"></div>
      <div class="col-sm-2 well" style="margin: 0px;">
        <h3>Total</h3>
        {{this.revenue_total}}
      </div>
      <div class="col-sm-2 well" style="margin: 0px;">
        <h3>US</h3>
        {{this.revenue_us}}
      </div>
      <div class="col-sm-2 well" style="margin: 0px;">
        <h3>States</h3>
        <div id="us-state-revenue{{prefix}}" class="no_loading_msg" pd_render_onload pd_options="route_load_us_state_revenue=true"><span>N/A</span></div>
      </div>
      <div class="col-sm-2 well" style="margin: 0px;">
        <h3>International</h3>
        {{this.revenue_foreign}}
      </div>
      <div class="col-sm-2"></div>
    </div>
    <div class="row">
      <div class="col-sm-12">
        <ul class="nav nav-tabs">
          <li>
            <a id="us-nav{{prefix}}" data-toggle="tab" href="#us-tab{{prefix}}">US
              <target pd_target="us-map{{prefix}}" pd_entity="pdf_us" pd_options="handlerId=mapView;valueFields=revenue;keyFields=us_state;aggregation=SUM;rowCount=500;timeseries=false;legend=true;rendererId=google;mapRegion=US;clusterby=;googlemapapikey=""" + self.google_maps_api_key + """">
                <pd_event_handler pd_script="self.handle_us_event(eventInfo)" pd_refresh="us-chart{{prefix}},us-filters{{prefix}},us-state-revenue{{prefix}}" />
              </target>
            </a>
          </li>
          <li>
            <a id="foreign-nav{{prefix}}" data-toggle="tab" href="#foreign-tab{{prefix}}">International
              <target pd_target="foreign-map{{prefix}}" pd_render_onload pd_entity="pdf_foreign" pd_options="handlerId=mapView;valueFields=revenue;keyFields=country_code;aggregation=SUM;rowCount=500;timeseries=false;legend=true;rendererId=google;mapRegion=world;clusterby=;googlemapapikey=""" + self.google_maps_api_key + """">
              </target>
            </a>
          </li>
        </ul>
      </div>
    </div>
  </div>
  <div class="tab-content container-fluid">
    <div id="us-tab{{prefix}}" class="row tab-pane tab-pane-dist fade in active">""" + self.get_us_tab_html() + """
    </div>
    <div id="foreign-tab{{prefix}}" class="row tab-pane tab-pane-dist fade in active">""" + self.get_foreign_tab_html() + """
    </div>
  </div>
</div>
<script>$('#us-nav{{prefix}}').click();</script>"""
    
    def get_us_tab_html(self):
        return """
<div class="row" style="padding:20px 0px 10px 0px;">
  <div id="us-filters{{prefix}}" class="no_loading_msg" pd_render_onload pd_options="route_load_us_filters=true">
  </div>
</div>
<div class="row" style="height:100%">
  <div class="col-sm-6">
    <div class="well" style="margin: 10px;">
      <div id="us-chart{{prefix}}" pd_render_onload pd_options="route_load_us_chart=true">
      </div>
    </div>
  </div>
  <div class="col-sm-6">
    <div class="well" style="margin: 10px;">
      <div id="us-map{{prefix}}">
      </div>
    </div>
  </div>
</div>
"""

    def get_us_filters_html(self):
        if self.pdf_us.empty:
            return ""
        else:
            states = sorted(self.pdf_us['us_state'].unique())
            state_options = ""
            state_buttons = ""
            for state in states:
                if state not in self.selected_states:
                    state_options += """<option value="{}">{}</option>""".format(state,state)
                elif state in self.selected_states:
                    state_buttons += """
<button type="button" class="btn btn-outline-primary" pd_script="self.remove_state('""" + state + """')" pd_refresh="us-chart{{prefix}},us-filters{{prefix}},us-state-revenue{{prefix}}">""" + state + """ &times;
</button>"""
            return self.get_us_toggle_chart_html() + """
<div class="col-sm-1">
  <select class="form-control" id="state{{prefix}}" pd_stop_propagation>""" + state_options + """</select>
</div>
<div class="col-sm-9">
  <button type="submit" class="btn btn-primary" pd_script="self.add_state('$val(state{{prefix}})')" pd_refresh="us-chart{{prefix}},us-filters{{prefix}},us-state-revenue{{prefix}}">Add
  </button>""" + state_buttons + """
</div>"""
        
    def get_us_toggle_chart_html(self):
        if len(self.selected_states) == 0:
            disabled = ' disabled'
        else:
            disabled = ''
        return """
<div class="col-sm-2" style="padding-right: 4px;">
  <select class="form-control" id="uschart{{prefix}}" pd_script="self.set_us_chart('$val(uschart{{prefix}})')" pd_refresh='us-chart{{prefix}}'""" + disabled + """>
    <option value='bar'""" + (' selected' if self.selected_us_chart == 'bar' else '') + """">Aggregated</option>
    <option value='line'""" + (' selected' if self.selected_us_chart == 'line' else '') + """>Over Time</option>
    <option value='3D'""" + (' selected' if self.selected_us_chart == '3D' else '') + """>3D Bar Chart</option>
  </select>
</div>"""
        
    def get_foreign_tab_html(self):
        return self.get_foreign_toggle_chart_html() + """
<div class="row" style="height:100%">
  <div class="col-sm-6">
    <div class="well" style="margin: 10px;">
      <div id="foreign-chart{{prefix}}" pd_render_onload pd_options="route_load_foreign_chart=true">
      </div>
    </div>
  </div>
  <div class="col-sm-6">
    <div class="well" style="margin: 10px;">
      <div id="foreign-map{{prefix}}">
      </div>
    </div>
  </div>
</div>
"""
    
    def get_foreign_toggle_chart_html(self):
        if self.pdf_foreign.empty:
            return ""
        else:
            return """
<div class="row" style="padding:20px 0px 10px 0px;">
  <div class="col-sm-2">
    <select class="form-control" id="foreignchart{{prefix}}" pd_script="self.set_foreign_chart('$val(foreignchart{{prefix}})')" pd_refresh="foreign-chart{{prefix}}">
      <option value='bar'""" + (' selected' if self.selected_foreign_chart == 'bar' else '') + """>Aggregated</option>
      <option value='line'""" + (' selected' if self.selected_foreign_chart == 'line' else '') + """>Over Time</option>
    </select>
  </div>
</div>"""
        
    def download_data_asset(self, access_token, asset_id, catalog_id):
        data_asset = get_data_asset(access_token, asset_id, catalog_id)
        if data_asset is None or 'attachments' not in data_asset.keys():
            return None
        attachments = data_asset['attachments']
        attachment = None
        if len(attachments) > 0:
            for a in attachments:
                if 'asset_type' in a.keys() and a['asset_type'] == 'data_asset':
                    attachment = a
                    break
        if attachment is None:
            return None
        attachment = attachments[0]
        connection_id = attachment['connection_id']
        # cache connection and connection access token based on connection id
        # if connection not in cache then load it and get an access key for the connection
        if connection_id in self.connection_cache.keys():
            connection = self.connection_cache[connection_id]['connection']
            connection_access_token = self.connection_cache[connection_id]['connection_access_token']
        else:
            self.info('Loading connection {0}'.format(connection_id))
            connection_access_token = None
            connection = get_connection(access_token, connection_id, catalog_id)
        if connection is None:
            return None
        connection_properties = connection['properties']
        if connection_access_token is None:
            connection_iam_url = connection_properties['iam_url']
            connection_api_key = connection_properties['api_key']
            connection_access_token = get_access_token(connection_iam_url, connection_api_key)
            if connection_access_token is None:
                return None
            else:
                self.connection_cache[connection_id] = {
                    'connection': connection,
                    'connection_access_token': connection_access_token
                }
        else:
            self.info('Using connection {0} from cache'.format(connection_id))
        connection_url = connection_properties['url']
        connection_path = attachment['connection_path']
        asset = download_file(connection_access_token, '{0}/{1}'.format(connection_url, connection_path))
        if asset is not None:
            return pandas.read_csv(io.StringIO(asset.content.decode('utf-8')))
        else:
            return None
        
app = CatalogDashboard()
app.run()

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!  

Check out other notebooks in this series: 
 - Part one: Static data analysis using Python and PixieDust
 - Part two: Build a product recommendation engine

### Author
Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.